<a href="https://colab.research.google.com/github/SergeyBryukhanov/SergeyBryukhanov/blob/main/gradio_interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Интерфейс Gradio для модели от SberDevice. В качестве данных - статья из википедии по шахматам.

In [1]:
%%writefile requirements.txt
transformers>=4.42.0
llama_index==0.10.46
pyvis==0.3.2
Ipython==7.34.0
langchain==0.2.5
pypdf==4.2.0
sentencepiece==0.1.99
accelerate==0.31.0
bitsandbytes==0.43.1
peft==0.11.1
llama-index-readers-wikipedia==0.1.4
wikipedia==1.4.0

# Зависимости
huggingface-hub==0.23.3
torch>=2.3.1
numpy==1.26
packaging==24.2
pyyaml==6.0.1
requests==2.32.3
tqdm==4.66.4
filelock==3.14.0
regex==2024.5.15
typing-extensions==4.14
safetensors==0.4.3
tokenizers==0.19.1
gradio

Writing requirements.txt


In [4]:
!pip install -r requirements.txt

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 4.1 MB/s eta 0:00:00
INFO: pip is still looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtra

In [1]:
from huggingface_hub import login
from getpass import getpass


HF_TOKEN = getpass("Введите ваш токен Hugging Face: ")
login(token=HF_TOKEN, add_to_git_credential=True)

Введите ваш токен Hugging Face: ··········
Token is valid (permission: fineGrained).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
import torch
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer


# Задаем имя модели
MODEL_NAME = "ai-forever/rugpt3large_based_on_gpt2"

# Загружаем LoRA модель
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

# Переводим модель в режим инференса
model.eval()

# Загружаем токенизатор
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/622 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.14G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

In [3]:
from llama_index.readers.wikipedia import WikipediaReader


def process_question(article, question):
    # Формируем промпт для модели
    max_input_length = 2000  # Максимальная длина входного текста (включая промпт и вопрос)
    truncated_article = article[:max_input_length - 200]  # Оставляем место для промпта и вопроса
    prompt = f"Статья: {truncated_article}\n\nВопрос: {question}\n\nОтвет:"

    # Генерация ответа
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,  # Ограничиваем длину ответа
        num_return_sequences=1,
        temperature=0.8,  # Управляет случайностью ответа
        top_p=0.9,  # Управляет разнообразием
    )
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return result


def load_wikipedia_article():
    # Инициализация объекта WikipediaReader
    reader = WikipediaReader()

    # Загрузка данных из википедии
    articles = reader.load_data(
        pages=["Шахматы"],  # запрос раздела на тему шахмат
        lang_prefix='ru'     # из русскоязычной зоны википедии
    )

    # Извлечение текста из первого документа
    if articles and len(articles) > 0:
        return articles[0].text  # Предполагаем, что текст статьи находится в атрибуте .text
    else:
        return "Статья не найдена."

In [4]:
import gradio as gr


# Создаём интерфейс Gradio
def gradio_interface():
    with gr.Blocks() as demo:
        # Поле для отображения статьи
        article_input = gr.Textbox(label="Статья из Википедии", lines=10, interactive=False)

        # Кнопка для загрузки статьи
        load_button = gr.Button("Распарсить данные")

        # Поле для ввода вопроса
        question_input = gr.Textbox(label="Отправить запрос", placeholder="Введите ваш вопрос...")

        # Кнопка для отправки запроса
        submit_button = gr.Button("Отправить запрос")

        # Поле для вывода результата
        output = gr.Textbox(label="Ответ модели", lines=10)

        # Обработка нажатия кнопки "Распарсить данные"
        load_button.click(fn=load_wikipedia_article, outputs=article_input)

        # Обработка нажатия кнопки "Отправить запрос"
        submit_button.click(fn=process_question, inputs=[article_input, question_input], outputs=output)

    # Запуск интерфейса
    demo.launch()

# Запускаем интерфейс
gradio_interface()

# Вопрос для модели: что такое шахматы?
# Запрос выполняется 1-2 минуты

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://6c7446f30930a8f058.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
